In [5]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.layers import DepthwiseConv2D
from tensorflow.keras.layers import Add
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, Reshape, GlobalAveragePooling2D, Activation,UpSampling2D, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools

from mediapipe.python.solutions import pose as mp_pose
import cv2
pose_tracker = mp_pose.Pose()

In [6]:
train_path = 'C:/Users/Prasanna P M/Human Project/ML_aug/Human Action Recognition/TRAIN/'
valid_path = 'C:/Users/Prasanna P M/Human Project/ML_aug/Human Action Recognition/VALID/'
test_path = 'C:/Users/Prasanna P M/Human Project/ML_aug/Human Action Recognition/TEST/'

train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=train_path, target_size=(224,224), batch_size=32)

valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=valid_path, target_size=(224, 224), batch_size=32)

test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=test_path, target_size=(224,224), batch_size=32, shuffle=False)

Found 17640 images belonging to 15 classes.
Found 5040 images belonging to 15 classes.
Found 2520 images belonging to 15 classes.


**Data Preprocessing**

In [7]:
import os
import cv2
import numpy as np
from tensorflow.keras.utils import Sequence
from tensorflow.keras.applications.mobilenet import preprocess_input
from mediapipe.python.solutions import pose as mp_pose
import tensorflow as tf

# class LandmarkDataGenerator(Sequence):
#     def __init__(self, image_paths, class_labels, batch_size=32, target_size=(224, 224)):
#         self.image_paths = image_paths
#         self.class_labels = class_labels
#         self.batch_size = batch_size
#         self.target_size = target_size
#         self.pose_tracker = mp_pose.Pose()

#     def __len__(self):
#         return int(np.ceil(len(self.image_paths) / self.batch_size))

#     def __getitem__(self, index):
#         batch_image_paths = self.image_paths[index * self.batch_size:(index + 1) * self.batch_size]
#         batch_class_labels = self.class_labels[index * self.batch_size:(index + 1) * self.batch_size]

#         images = []
#         landmarks = []

#         for image_path in batch_image_paths:
#             image = cv2.imread(image_path)
#             images.append(image)

#             # Process the image to extract landmarks
#             results = self.pose_tracker.process(image)
#             pose_landmarks = results.pose_landmarks

#             if pose_landmarks is not None:
#                 landmark_coordinates = []
#                 for landmark in pose_landmarks.landmark:
#                     landmark_coordinates.append([landmark.x, landmark.y, landmark.z])
#                 landmarks.append(landmark_coordinates)
#             else:
#                 landmarks.append([])

#         images = np.array(images)
#         landmarks = np.asarray(landmarks).astype(np.float32)  # Convert landmarks to a tensor
#         class_labels = np.array(batch_class_labels)

#         return images, landmarks, class_labels


# # Rest of the code...





In [8]:

# train_data_generator = LandmarkDataGenerator(train_batches.filepaths, train_batches.classes)

# # Create the custom data generator for validation
# valid_data_generator = LandmarkDataGenerator(valid_image_paths, valid_class_labels)

# # Create the custom data generator for testing
# test_data_generator = LandmarkDataGenerator(test_image_paths, test_class_labels)

# # Example of iterating over the data generator
# for [images, landmarks], class_labels in train_data_generator:
#     # Perform model training or evaluation using the images, landmarks, and class labels
# #     ...
# train_batches.classes

In [9]:
# images = train_data_generator[0][0]  # Get the images from the first batch

# # Accessing landmarks
# landmarks = train_data_generator[0][1]  # Get the landmarks from the first batch

# # Accessing class labels
# class_labels = train_data_generator[0][2]

In [10]:
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, Dense, Flatten
# from tensorflow.keras.applications.mobilenet import MobileNet

# # Define the input layers for images and landmarks
# image_input = Input(shape=(224, 224, 3))
# landmark_input = Input(shape=(33, 3))

# # Pretrained MobileNet model for images
# base_model = MobileNet(input_tensor=image_input, include_top=False, weights='imagenet')
# image_features = base_model.output
# image_features = Flatten()(image_features)

# # Flatten the landmark input
# landmark_features = Flatten()(landmark_input)

# # Concatenate image and landmark features
# merged_features = tf.keras.layers.concatenate([image_features, landmark_features])

# # Add a dense layer for classification
# dense_layer = Dense(256, activation='relu')(merged_features)
# output_layer = Dense(15, activation='softmax')(dense_layer)

# # Create the model
# model = Model(inputs=image_input, outputs=output_layer)

# # Compile the model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Create an instance of the LandmarkDataGenerator
# train_data_generator = LandmarkDataGenerator(train_batches.filepaths, train_batches.classes)

# # Fit the model
# model.fit(train_data_generator, epochs=15)


**Data Prcoessing 2**

**Model building**

In [25]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dropout, Dense, concatenate
from tensorflow.keras.models import Model

model1 = tf.keras.Sequential([
    hub.KerasLayer(
        'https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/feature_vector/5',
        trainable=True),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(128, activation='relu'),
])

# Build the model
model1.build((None, 224, 224, 3))
model1.summary()




Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_3 (KerasLayer)   (None, 1280)              4226432   
_________________________________________________________________
dropout_5 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               163968    
Total params: 4,390,400
Trainable params: 4,366,000
Non-trainable params: 24,400
_________________________________________________________________


In [26]:


# Define MobileNetV2 model
mobile = tf.keras.applications.MobileNetV2(
    input_shape=(32, 32, 3), weights=None, include_top=False)


In [29]:
x = mobile.layers[-1].output
x = GlobalAveragePooling2D()(x)  # Global Average Pooling to reduce dimensions to (None, 1)
x = Dropout(0.1)(x)
output = Dense(128, activation='relu')(x)

model2 = Model(inputs=mobile.input, outputs=output)
model2.summary()



Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 16, 16, 32)   864         input_4[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 16, 16, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 16, 16, 32)   0           bn_Conv1[0][0]                   
____________________________________________________________________________________________

In [34]:
combined = concatenate([model1.output,model2.output])
prediction = Dense(15, activation='softmax')(combined)

fin_model = Model(inputs=[model1.input,model2.input], outputs=prediction)
fin_model.summary()

Model: "model_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 16, 16, 32)   864         input_4[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 16, 16, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 16, 16, 32)   0           bn_Conv1[0][0]                   
___________________________________________________________________________________________

In [35]:
fin_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy'])


In [37]:
train_generator = data_generator(train_dir, batch_size)

# Retrieve and print the first batch
batch_images, batch_landmarks, batch_labels = next(train_generator)
print("Batch Images:", batch_images.shape)
print("Batch Landmarks:", batch_landmarks.shape)
print("Batch Labels:", batch_labels.shape)

# Feed the model with the batch
fin_model.fit([batch_images, batch_landmarks], batch_labels, epochs = 5,verbose = 1)

Batch Images: (32, 224, 224, 3)
Batch Landmarks: (32, 32, 32, 3)
Batch Labels: (32, 15)
Epoch 1/5
1/1 [==============================] - 5s 5s/step - loss: 2.2597 - accuracy: 0.3750
Epoch 2/5
1/1 [==============================] - 5s 5s/step - loss: 1.5218 - accuracy: 0.9062
Epoch 3/5
1/1 [==============================] - 4s 4s/step - loss: 1.0376 - accuracy: 1.0000
Epoch 4/5
1/1 [==============================] - 4s 4s/step - loss: 0.6845 - accuracy: 1.0000
Epoch 5/5
1/1 [==============================] - 4s 4s/step - loss: 0.4913 - accuracy: 1.0000


In [11]:
import os
import shutil
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from mediapipe.python.solutions import pose as mp_pose

train_dir = "C:/Users/Prasanna P M/Human Project/ML_aug/Human Action Recognition/TRAIN/"
batch_size = 32

# Initialize the pose tracker
pose_tracker = mp_pose.Pose()

# Create a dictionary to map subclass folders to class labels
subclass_to_class = {
    "sitting": 0,
    "sleeping": 1,
    "texting": 2,
    "using_laptop": 3,
    "running": 4,
    "listening_to_music": 5,
    "laughing": 6,
    "fighting": 7,
    "hugging": 8,
    "eating": 9,
    "drinking": 10,
    "dancing": 11,
    "clapping": 12,
    "cycling": 13,
    "calling": 14
}

def data_generator(train_dir, batch_size):
    # Initialize the pose tracker
    pose_tracker = mp_pose.Pose()

    while True:
        images = []
        landmarks = []
        class_labels = []

        # Iterate over the source directory and process images and landmarks
        for root, dirs, files in os.walk(train_dir):
            for file in files:
                if file.endswith(".jpg") or file.endswith(".png"):
                    train_file = os.path.join(root, file)

                    # Process the image to extract landmarks
                    image = cv2.imread(train_file)
                    results = pose_tracker.process(image)
                    pose_landmarks = results.pose_landmarks

                    # Extract landmarks and perform further processing if landmarks are found
                    if pose_landmarks is not None and len(pose_landmarks.landmark) == 33:
                        # Apply preprocessing to the image
                        preprocessed_image = tf.keras.applications.mobilenet.preprocess_input(image)

                        # Store the image, landmarks, and class label
                        images.append(preprocessed_image)
                        landmark_coordinates = [[lmk.x, lmk.y, lmk.z] for lmk in pose_landmarks.landmark]
                        landmarks.append(landmark_coordinates)

                        subclass_folder = os.path.basename(os.path.dirname(train_file))
                        class_label = subclass_to_class.get(subclass_folder)
                        class_labels.append(class_label)

                if len(images) == batch_size:
                    # Convert target labels to one-hot encoded format
                    class_labels = to_categorical(class_labels, num_classes=15)

                    # Convert landmarks to a NumPy array
                    landmarks = np.array(landmarks)

                    # Append zeros and reshape landmarks array
                    landmarks = np.concatenate((landmarks, np.zeros((batch_size, 1024 - 33, 3))), axis=1)
                    landmarks = landmarks.reshape((batch_size, 32, 32, 3))

                    yield (np.array(images), landmarks, np.array(class_labels))
                    images = []
                    landmarks = []
                    class_labels = []

        if len(images) > 0:
            # Convert target labels to one-hot encoded format
            class_labels = to_categorical(class_labels, num_classes=15)

            # Convert landmarks to a NumPy array
            landmarks = np.array(landmarks)

            # Append zeros and reshape landmarks array
            landmarks = np.concatenate((landmarks, np.zeros((len(images), 1024 - 33, 3))), axis=1)
            landmarks = landmarks.reshape((len(images), 32, 32, 3))

            yield (np.array(images), landmarks, np.array(class_labels))

         
train_generator = data_generator(train_dir, batch_size)

In [12]:
valid_dir = "C:/Users/Prasanna P M/Human Project/ML_aug/Human Action Recognition/VALID/"
batch_size = 32
    

valid_generator = data_generator(valid_dir, batch_size)


In [13]:
test_dir = "C:/Users/Prasanna P M/Human Project/ML_aug/Human Action Recognition/TEST/"
batch_size = 32
    

test_generator = data_generator(test_dir, batch_size)


In [14]:
train_generator = data_generator(train_dir, batch_size)

# Retrieve and print the first batch
batch_images, batch_landmarks, batch_labels = next(train_generator)
print("Batch Images:", batch_images.shape)
print("Batch Landmarks:", batch_landmarks.shape)
print("Batch Labels:", batch_labels.shape)

# Retrieve and print the second batch
batch_images, batch_landmarks, batch_labels = next(train_generator)
print("Batch Images:", batch_images.shape)
print("Batch Landmarks:", batch_landmarks.shape)
print("Batch Labels:", batch_labels.shape)

# ... continue retrieving and printing batches as needed


Batch Images: (32, 224, 224, 3)
Batch Landmarks: (32, 32, 32, 3)
Batch Labels: (32, 15)
Batch Images: (32, 224, 224, 3)
Batch Landmarks: (32, 32, 32, 3)
Batch Labels: (32, 15)
